<a href="https://colab.research.google.com/github/Fabricio-emiliano/PySpark-no-Google-Collab/blob/main/PySpark_Quick_Start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

# instalação do pyspark no google collab:
# ps só roda uma vez

# Instalar Java, Spark e Findspark
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
# !tar xf spark-3.0.0-bin-hadoop3.2.tgz
# !pip install -q findspark

# # Configurar variáveis de ambiente
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

# import findspark
# findspark.init()

# # Iniciar uma sessão Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("QuickStartPySpark").master("local[*]").getOrCreate()



In [13]:
# Pacotes
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
from pyspark.sql.window import Window
from pyspark.sql.functions import col,lag
from pyspark.sql import functions as F
import requests
import plotnine as pn


In [7]:
url = "https://olinda.bcb.gov.br/olinda/servico/Pix_DadosAbertos/versao/v1/odata/ChavesPix(Data=@Data)?@Data='2024-01-01'&$format=json"

response = requests.get(url)


In [52]:
dados_json= response.json()['value']
df_spark = spark.createDataFrame(dados_json)

df_spark = df_spark.where(df_spark.NaturezaUsuario=='PF')

df_spark.orderBy(["ISPB", "Data",'qtdChaves'], ascending=[False, False,False]).show()
# Dá para perceber que há uma variação de uma data para outra, podemos realizar uma análise de elasticidade cruzada (canibalização)
# para entender quando um usuário sai de um banco, para onde ele vai em seguida?

# a ideia é entender a variação. posso rodar um código de analise de correlação direto, mas pode haver algum erro,
# já que pode haver uma tendencia de crescimento total de chaves pix.

+----------+--------+---------------+--------------------+---------+---------+
|      Data|    ISPB|NaturezaUsuario|                Nome|TipoChave|qtdChaves|
+----------+--------+---------------+--------------------+---------+---------+
|2024-10-31|97489280|             PF|COOP SICREDI INTE...|Aleatória|     2973|
|2024-10-31|97489280|             PF|COOP SICREDI INTE...|  Celular|     1801|
|2024-10-31|97489280|             PF|COOP SICREDI INTE...|      CPF|     1645|
|2024-10-31|97489280|             PF|COOP SICREDI INTE...|   e-mail|     1220|
|2024-09-30|97489280|             PF|COOP SICREDI INTE...|Aleatória|     2860|
|2024-09-30|97489280|             PF|COOP SICREDI INTE...|  Celular|     1758|
|2024-09-30|97489280|             PF|COOP SICREDI INTE...|      CPF|     1639|
|2024-09-30|97489280|             PF|COOP SICREDI INTE...|   e-mail|     1185|
|2024-08-31|97489280|             PF|COOP SICREDI INTE...|Aleatória|     2714|
|2024-08-31|97489280|             PF|COOP SICREDI IN

In [53]:
variacao_chaves = df_spark.groupBy('Data').sum('qtdChaves').withColumnRenamed("sum(qtdChaves)", "total_chaves")
variacao_chaves.show()


+----------+------------+
|      Data|total_chaves|
+----------+------------+
|2024-04-30|   705791757|
|2024-07-31|   740588717|
|2024-08-31|   750398340|
|2024-05-31|   716953230|
|2024-09-30|   760539978|
|2024-01-31|   676707499|
|2024-02-29|   687306013|
|2024-10-31|   766354219|
|2024-06-30|   728396899|
|2024-03-31|   696952729|
+----------+------------+



In [55]:
# Definir a janela
window_spec = Window.orderBy("Data")

# Definir o valor inicial da data mais antiga:
valor_inicial = variacao_chaves.orderBy('Data',ascending=True).select('total_chaves').first()
valor_inicial = valor_inicial[0]

# Calcular a diferença entre a quantidade atual e a quantidade da linha anterior
variacao_chaves = variacao_chaves.withColumn("total_diff", ((col("total_chaves") - valor_inicial)/valor_inicial))

# variacao_chaves = variacao_chaves.na.fill(0)

variacao_chaves = variacao_chaves.withColumn("diferenca", col("total_chaves")*(1-col('total_diff')) )

variacao_chaves = variacao_chaves.orderBy('Data',ascending=True)
# Exibir os dados com a diferença
variacao_chaves.show()

# Estamos interessados na coluna total_diff_cumsum. Ela que vai retirar o fator de crescimento das chaves


+----------+------------+--------------------+-------------------+
|      Data|total_chaves|          total_diff|          diferenca|
+----------+------------+--------------------+-------------------+
|2024-01-31|   676707499|                 0.0|       6.76707499E8|
|2024-02-29|   687306013| 0.01566188348091588|6.765415063086611E8|
|2024-03-31|   696952729|0.029917253805990408|6.761018173157294E8|
|2024-04-30|   705791757| 0.04297906856799883|6.754574846811687E8|
|2024-05-31|   716953230|0.059472860962044696|6.743139702359211E8|
|2024-06-30|   728396899|  0.0763836666157589|6.727592731028314E8|
|2024-07-31|   740588717| 0.09440004447179919|6.706771091798873E8|
|2024-08-31|   750398340| 0.10889614953121718|6.686828501593828E8|
|2024-09-30|   760539978| 0.12388288754577552|6.663220894313594E8|
|2024-10-31|   766354219|   0.132474843462611|6.648315638010634E8|
+----------+------------+--------------------+-------------------+

